<a href="https://colab.research.google.com/github/SAJAD-52/CODSOFT/blob/main/Credit_Card_Transactions_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
data = pd.read_csv('fraudTrain.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
#visualization
sns.set(style='whitegrid')
sns.countplot(x='is_fraud', data=data)
sns.despine()
plt.title('Distribution of Fraudulent vs Legitimate Transactions')
plt.show()

In [ ]:
sns.countplot(x='gender', data=data)
plt.title('count of gender')
plt.show()

In [ ]:
sns.countplot(data, x='gender', hue='is_fraud' )
plt.title('fraud distrubtion by gender')
plt.show()

In [ ]:
# Get the top N cities by frequency
top_cities = data['city'].value_counts().head(10).index  # Change 10 to the desired number of cities
print(top_cities)

In [ ]:
# Filter the data to include only the top cities
data_top_cities = data[data['city'].isin(top_cities)]
data_top_cities.head()

In [ ]:
sns.countplot(data=data_top_cities, y='city', hue='is_fraud')
plt.ylabel('city')
plt.xlabel('count')
plt.title('fraud by city')
plt.show()

In [ ]:
# Plot relationship between transaction category and fraud
#plt.figure(figsize=(12, 8))
#sns.countplot(y='category', hue='is_fraud', data=data_to_amt)
#sns.despine()
#plt.title('Relationship between Transaction Category and Fraud')
#plt.ylabel('Category')
#plt.xlabel('Frequency')
#plt.show()

In [ ]:
# Get the top 20 amounts where fraud is equal to 1
fraudulent_transactions = data[data['is_fraud'] == 1]
# Filter the data to include only the rows with the top 20 amounts where fraud is equal to 1
data_top_amt_fraudulent = fraudulent_transactions['amt'].value_counts().head(20).index
# Filter the data to include only the rows with the top 20 amounts from fraudulent transactions
data_top_amt_fraudulent = fraudulent_transactions[fraudulent_transactions['amt'].isin(data_top_amt_fraudulent)]

In [ ]:
fraudulent_transactions.head()

In [ ]:
# Sort the data by the count of fraudulent transactions for each amount in descending order
sorted_amt = data_top_amt_fraudulent.groupby('amt')['is_fraud'].sum().sort_values(ascending=False).index

# Plot 20 top amount by fraud count
plt.figure(figsize=(12, 8))
sns.countplot(y='amt', hue='is_fraud', order=sorted_amt, data=data_top_amt_fraudulent )
sns.despine()
plt.title('Relationship between Transaction Category and Fraud')
plt.ylabel('Category')
plt.xlabel('Frequency')
plt.show()

In [ ]:
#Features selection
data_dropped = data.drop(['Unnamed: 0'], axis=1)
# Select only numeric columns for feature selection
numeric_columns = data_dropped.select_dtypes(include=['int64', 'float64']).columns
corr = data_dropped[numeric_columns].corr()

In [ ]:
#plot the  correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
data.info()

In [ ]:
from scipy.stats import chi2_contingency
# Assuming your original DataFrame is named 'data'
categorical_features = data.select_dtypes(include=['object']).copy()
# Iterate over each categorical feature
for column in categorical_features.columns:
    # Contingency table between the current categorical feature and the target variable
    contingency_table = pd.crosstab(categorical_features[column], data['is_fraud'])

    # Chi-square test
    chi2, p_value, _, _ = chi2_contingency(contingency_table)

    # Print results
    print("Feature:", column)
    print("Chi-square value:", chi2)
    print("P-value:", p_value)
    print("\n")

In [ ]:
#lets use start classifer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder

data.dropna(subset=['merchant', 'category', 'street', 'city', 'job', 'dob', 'state', 'amt','is_fraud'], inplace=True)


X = data[['merchant', 'category', 'street', 'city', 'job', 'dob', 'state', 'amt']]
X_train = X.iloc[:, :-1]
Y_train = data['is_fraud']

onecoder = OneHotEncoder()
X_encoded = onecoder.fit_transform(X_train)


In [ ]:
data_test = pd.read_csv('Copie de Copie de fraudTest.csv', on_bad_lines='skip')
data_test.head()


In [ ]:
data_test.dropna(inplace=True)

In [ ]:
data_test.describe()

In [ ]:
Xt= data_test[['merchant', 'category', 'street', 'city', 'job', 'dob', 'state', 'amt']]
Xtest = Xt.iloc[:, :-1]
Ytest = data_test['is_fraud']

X_test_encoded = onecoder.fit_transform(Xtest)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()


RFC.fit(X_test_encoded, Ytest)
pred_y = RFC.predict(X_test_encoded)

In [ ]:
# Print classification report
print(classification_report(Ytest, pred_y))

In [ ]:
LRC = LogisticRegression()
LRC.fit(X_test_encoded, Ytest)
pred_y2 = LRC.predict(X_test_encoded)

In [ ]:
# Print classification report
print(classification_report(Ytest, pred_y2))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier()

DTC.fit(X_test_encoded, Ytest)
y_pred3 = DTC.predict(X_test_encoded)

In [ ]:
print(classification_report(Ytest, y_pred3))